In [56]:
from copy import deepcopy
import math
from pathlib import Path

from arcgis.features import FeatureSet, FeatureLayerCollection
from arcgis.geometry import SpatialReference
from arcgis.gis import GIS, CreateServiceParameter
import pandas as pd

from reach_tools import Reach

In [35]:
raw_dir_pth = Path.cwd().parent / f"data/raw/american_whitewater/"
url = 'https://adventure.maps.arcgis.com'
arcgis_client_id = "2MonB9785a7oRJEl"

In [17]:
# get the list of all downloaded reach json files
raw_json_lst = sorted(list(raw_dir_pth.glob("aw_*.json")))

In [18]:
# get a dictionary of the maximum lengths for all the string columns plus 10 percent rounded up to the next whole integer
dict_lst = [{k: len(v) if isinstance(v, str) else None for k, v in Reach.from_aw_json(reach_json).attributes.items()} for reach_json in raw_json_lst]

len_dict = pd.DataFrame.from_records(dict_lst).max().dropna().astype(int).apply(lambda val: math.ceil(val * 1.1)).astype(int).to_dict()

In [29]:
# get a list of features as dictionary objects
feature_lst = [Reach.from_aw_json(reach_json).line_feature.as_dict for reach_json in raw_json_lst]

In [74]:
# creeate a feature set from the list of dictionary objects
fs_prop = FeatureSet.from_dict({
    "features": [feature_lst[0]],
    "spatialReference": {"wkid": 4326}
})

# add length to the feature set field if a length was found above
def add_length(fld: dict) -> dict:
    fld_len = len_dict.get(fld['name'])
    if fld_len is not None:
        fld['length'] = fld_len
    return fld

fs_prop.fields = [add_length(fld) for fld in fs_prop.fields]

In [88]:
gis = GIS(url, client_id=arcgis_client_id)

gis

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://adventure.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=2MonB9785a7oRJEl&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=axGLAP0lwcaD0ID1hbJ3Q48Hn4HD5R&allow_verification=false


Enter code obtained on signing in using SAML:  ········


GIS @ https://adventure.maps.arcgis.com version:2025.1

In [89]:
service_name = 'test_feature_service'

In [90]:
existing_lst = gis.content.search(service_name)

if len(existing_lst) > 0:
    gis.content.delete_items(existing_lst)

new_svc_itm = gis.content.create_service('test_feature_service', service_type='featureService', wkid=4326)

new_svc_itm

<Item title:"test_feature_service" type:Feature Layer Collection owner:jmccune_personal>

In [91]:
flc = FeatureLayerCollection.fromitem(new_svc_itm)

In [92]:
fs_dict = fs_prop.to_dict()

In [93]:
fs_dict = {k: v for k, v in fs_dict.items() if k != 'features'}

In [94]:
fs_dict["uniqueField"] =  {"name" : "OBJECTID", "isSystemMaintained" : True}

In [95]:
flc.manager.add_to_definition(fs_dict)

{'success': True}